# Generate the data for the ISDM-24 track

In [ ]:
%pip install -q pandas openpyxl jinja2

In [ ]:
# Print formatted dates in spanish locale
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

In [ ]:
# Read the dataframe from the .xlsx file given by Easychair
import pandas as pd
df = pd.read_excel('../info/ISDM2024.xlsx',index_col=0,header=0)

# df.Author to spanish 'and' -> 'y'
df['Authors'] = df['Authors'].str.replace(' and ', ' y ', regex=False)
df

In [ ]:
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class Session:
    name: str
    start: datetime
    paperstart: datetime
    end: datetime
    location: str
    chair: str
    papers: list[int]

isdm1 = Session('ISDM-1',
                datetime(2024,7,18,9,15),
                datetime(2024,7,18,9,15)+timedelta(minutes=10), # 10 mins presentation
                datetime(2024,7,18,10,15),
                'Aula A.3.1',
                'Diego Sevilla',
                [91, 94, 24])

isdm2 = Session('ISDM-2',
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,17,30),
                'Aula A.3.1',
                'Pedro J. Clemente',
                [3, 2, 4, 121, 124])

isdm3 = Session('ISDM-3',
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,19,00),
                'Aula A.3.1',
                'Diego Sevilla y Pedro J. Clemente',
                [12, 57, 81])

sessions = {s.name : s for s in [isdm1, isdm2, isdm3]}

# No article left behind
assert set(df.index) == set().union(*list(map(lambda s: s.papers, sessions.values())))

In [ ]:
# Produce the slide index from the slideindex template
from jinja2 import Template

def gen_session_index(sessions: dict[str, Session]) -> None:
    template = Template(open('templates/sessionindex.tex.j2').read())
    with open('../pr/sessionindex.tex', 'w') as f:
        f.write(template.render(sessions=sessions))

gen_session_index(sessions)

In [ ]:
def gen_session_slides(sessions: dict[str, Session], df: pd.DataFrame) -> None:
    template = Template(open('templates/sessionslide.tex.j2').read())

    def get_duration_time(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return 20
            case 'corto' | 'herramienta':
                return 10
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def get_duration(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return f'{get_duration_time(articletype)}~min'
            case 'corto' | 'herramienta':
                return f'{get_duration_time(articletype)}~min'
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def time_str(d: datetime) -> str:
        return d.strftime('%H:%M')

    def date_str(d: datetime) -> str:
        return d.strftime('%A, %d de %B de %Y, %H:%M')

    for session_name, session in sessions.items():
        with open(f'../pr/{session_name}.tex', 'w') as f:
            f.write(template.render(session=session,
                                    papers=session.papers,
                                    date_str = date_str,
                                    time_str = time_str,
                                    df=df,
                                    get_duration=get_duration,
                                    get_duration_time=get_duration_time,
                                    timedelta=timedelta))

gen_session_slides(sessions, df)

In [ ]:
%pip install -q pyecore pyecoregen

In [ ]:
!pyecoregen -e isdm-track/model/IsdmTrack.ecore -o . --auto-register-package

In [ ]:
import IsdmTrack as isdm

track = isdm.Track(name="ISDM'24")

# Participants
i_partiipants: dict[str, isdm.Participant] = {}

# Papers and authors, traverse the df DataFrame
i_papers: dict[int, isdm.Paper] = {}
for index,row in df.iterrrows():
    i_papers[index] = isdm.Paper(title=row['Title'],
                                 tipo=row['Tipo'],
                                 abstract=row['Abstract'],
                                 keywords=row['Keywords'])
    # Split and add authors
    authors = row['Authors']
    authors = authors.replace(' y ', ',')
    authors = authors.split(',')
    for author_name in authors:
        author_name = author_name.strip()
        author = isdm.Participant(name=author_name)
        i_participants[author_name] = author
        i_papers[index].authors.append(author)


i_sessions: dict[str, isdm.Session] = {}

for name, session in sessions.items():
    i_sessions[name] = isdm.Session(
        name=session.name,
        start=int(round(session.start.timestamp())),
        end=int(round(session.end.timestamp())),
        paperstart=int(round(session.paperstart.timestamp())),
        location=session.location
    )


